# 0. Импорт и загрузка данных

In [ ]:
!pip install 'scikit-mobility==1.3.1' 'geopandas==1.1.0'

## 0.1 Импорт

In [ ]:
import numpy as np
import osmnx as ox
import random

from changer_migration.model import MigrationFlowModel
from changer_migration.planter import *
from changer_migration.ueqi import count_change_capacity

## 0.2 Функции

In [ ]:
# Получение значений UEQI для города
def get_ueqi(wff, name):
    city = wff.cities[wff.cities['region_city'].str.contains(name)]
    return city[
                ['region_city',
                "ueqi_residential",
                "ueqi_street_networks",
                "ueqi_green_spaces",
                "ueqi_public_and_business_infrastructure",
                "ueqi_social_and_leisure_infrastructure",
                "ueqi_citywide_space"]
            ].round(3)

## 0.3 Загрузка и подготовка данных

### 0.3.1 Данные из и для TownsNet

In [ ]:
# 0) Входные пути
REGION_PKL = "data/lo_region.pickle"        # pickle с Region
MATRIX_DIR = "data/provision"               # где лежат *_links.parquet
SEED = 42

region = MigrationFlowModel.from_pickle(REGION_PKL, seed=SEED)

In [ ]:
supporting_cities = gpd.read_parquet("data/1_polygons.parquet") # Опорные города ЛО
anchor_settlement = pd.read_csv('data/anchor_settlement.csv') # Города/деревни ЛО
ueqi_df = region.calculate_ueqi() # Расчет UEQI по новой формуле

### 0.3.2 Первичное обновление UEQI для Planter
потому что мы не знаем как подсчитаны старые UEQI для городов

In [ ]:
# Загрузка модели
wff = WorkForceFlows.from_pickle('data/wff_1812 new.pkl')

In [ ]:
ueqi_df[ueqi_df.town_name == 'Бокситогорск']

In [ ]:
name = "Ленинградская область, Бокситогорск" # Можно узнать все города из wff.cities

# Можно взять параметры из ueqi_df[ueqi_df.town_name == 'Мурино'] ← ВАЖНО! Убедитесь, что вы используете правильное название города без области, но для wff с областью
new_params = { # Первичное обновление UEQI для Planter
    "ueqi_residential":                         49.994,
    "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        46.869,
    "ueqi_public_and_business_infrastructure":  100.0,
    "ueqi_social_and_leisure_infrastructure":   49.994,
    "ueqi_citywide_space":                      9.374,
}

wff.update_city_params(name, new_params)
wff.recalculate_after_update()

In [ ]:
# Выделение городов только по ЛО и сохранение новых значений постоянного населения
area = ox.geocode_to_gdf("Ленинградская область")
lo_cities = wff.cities.clip(area.to_crs(3857)).copy()
lo_cities['population'] = lo_cities['population'] + lo_cities['flows_in'] - lo_cities['flows_out']
lo_cities['region_city'] = lo_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
lo_cities[['region_city','population']].to_csv("data/population.csv", index=False)

### 0.2.3 Обновление населения

In [ ]:
# update_population(region.model, towns_df, path='data/population.csv')

population = pd.read_csv('data/population.csv') 
region.update_population(population)

ueqi_df = region.calculate_ueqi()
region[121] # Проверим Бокситогорск

### ====== Принимаем это за базовое население ====== ###

# 1. Выбор города

In [ ]:
name = 'Бокситогорск'
get_ueqi(wff, name) # Просмотр нынешных значений UEQI для города

# 2. Изменение UEQI

In [ ]:
new_params = {
    # "ueqi_residential":                         49.994,
    # "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        56.869, # +10 UEQI
    # "ueqi_public_and_business_infrastructure":  100.0,
    # "ueqi_social_and_leisure_infrastructure":   49.994,
    # "ueqi_citywide_space":                      9.374,
}

# Update the city data in the DataFrame
wff.update_city_params('Ленинградская область, Бокситогорск', new_params)
wff.recalculate_after_update()

# Выделение области
area = ox.geocode_to_gdf("Ленинградская область")
highlighted_cities = wff.cities.clip(area.to_crs(3857)).copy()
highlighted_cities['population'] = highlighted_cities['population'] + highlighted_cities['flows_in'] - highlighted_cities['flows_out']
highlighted_cities['region_city'] = highlighted_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
highlighted_cities[['region_city', 'population']].to_csv("data/population_new.csv", index=False)

In [ ]:
# Обновим население городов после изменения UEQI
population_new = pd.read_csv('data/population_new.csv') 
region.update_population(population_new)
region[121] # Проверим Бокситогорск

### ====== Принимаем это за новое население ====== ###

# 3. Расчет матриц переходов по новым UEQI

## 3.1 Расчет количества и добавление необходимого сервиса

In [ ]:
# Посмотим, на сколько нужно поднять
change_capacity = count_change_capacity(region.towns, name, 10)
print(f"Изменение capacity на 10% для города {name}: {change_capacity} мест")

In [ ]:
updated_service = region.update_service(name, change_capacity, new_params)
print(f'Случайный обновленный сервис: {updated_service}')

print('======= Таблица для проверки новых сервисов =======')
region.services[(region.services.town.str.contains(name)) & (region.services.service_type == updated_service)]

## 3.2 Расчет матриц переходов

In [ ]:
region.calculate_provision([updated_service])

# 4. Расчет данных для городов

## 4.1 Чтение матриц переходов

In [ ]:
# Для сравнения карты визуально нужно запустить разделы 4.1-5 со старым и новым населением и 2 раза запустить fmap (в разных ячейках)
UPDATED = True                              # брать обновлённые из data/provision/updated

if UPDATED:
    population = pd.read_csv('data/population_new.csv') 
    region.update_population(population)
else:
    population = pd.read_csv('data/population.csv') 
    region.update_population(population)

region.load_migration_matrix(['RECREATION'], 
                            matrix_dir='data/provision',
                            use_updated=UPDATED, # Внимание! Если True, использовать новое население. Если False, использовать базовое.
                            average=True,
                            anchors=anchor_settlement)

In [ ]:
mobility = region.analyze_mobility(anchor_threshold=90)

# 5. Генерация карты миграции

In [ ]:
fmap = region.create_map()
fmap